### **Default Predict model Demo**  
---

In [ ]:
pip install --upgrade pip setuptools wheel

In [ ]:
pip install pandas pandas_profiling scikit-learn==0.24.2 mlxtend ipywidgets seldon-core boto3 shap

In [ ]:
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_validate
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
from math import log as log
import os
import mlflow
# Отключение ворнингов с самоподписными сертификатами для demo стенда
import urllib3
urllib3.disable_warnings()


pd.options.mode.chained_assignment = None

# этот блок закомментирован так как используется только на kaggle
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))
#PATH_to_file = '/kaggle/input/sf-dst-scoring/'

# # # этот блок закомментирован так как используется только локальной машине
from importlib import reload
print(os.listdir('./data'))
PATH_to_file = './data/'

In [ ]:
import utils_module28072020 as utils

In [ ]:
RANDOM_SEED = 42
!pip freeze > requirements.txt
CURRENT_DATE = pd.to_datetime('11/08/2020')

## 2. Импорт данных

In [ ]:
df_train = pd.read_csv(PATH_to_file+'train.csv')
df_test = pd.read_csv(PATH_to_file+'test.csv')
pd.set_option('display.max_columns', None)
print('Размерность тренировочного датасета: ', df_train.shape)
display(df_train.head(2))
print('Размерность тестового датасета: ', df_test.shape)
display(df_test.head(2))

In [ ]:
# ВАЖНО! для корректной обработки признаков объединяем трейн и тест в один датасет
df_train['Train'] = 1 # помечаем где у нас трейн
df_test['Train'] = 0 # помечаем где у нас тест

df = df_train.append(df_test, sort=False).reset_index(drop=True) # объединяем
#!Обратите внимание объединение датасетов является потенциальной опасностью для даталиков

In [ ]:
 # временной ряд (1)
time_cols = ['app_date']
# бинарные переменные (default не включаем в список) (5+1 = 6)
bin_cols = ['sex', 'car', 'car_type', 'good_work', 'foreign_passport']
# категориальные переменные (Train не включаем в список, так как мы сами его добавили) (3+1=4)
cat_cols = ['education', 'region_rating', 'home_address', 'work_address', 'sna', 'first_time']
# числовые переменные, client_id исключили из списка (8)
num_cols = ['age','decline_app_cnt','score_bki','bki_request_cnt','income','days']
# client_id не включаем в списки

### Приведение признаков к целевому виду

In [ ]:
df['age'] = np.log(df['age'] + 1)
df['decline_app_cnt'] = np.log(df['decline_app_cnt'] + 1)
df['bki_request_cnt'] = np.log(df['bki_request_cnt'] + 1)
df['income'] = np.log(df['income'] + 1)
df['education'] = df['education'].fillna('SCH')
df.app_date = pd.to_datetime(df.app_date, format='%d%b%Y')

start = df.app_date.min()
end = df.app_date.max()
df['days'] = (df.app_date - start).dt.days.astype('int')

In [ ]:
df

---
### Оценка корреляций

In [ ]:
utils.simple_heatmap('Матрица корреляции тренировочного датасета на числовых переменных',df[df['Train']==1], num_cols+['default'], 1.1, 1, 9)

***Резюме*** - сильно скорелированных между собой признаков нет, все берем в работу

### Значимость непрерывных переменных по ANOVA F test

In [ ]:
temp_df = df[df['Train']==1]
imp_num = pd.Series(f_classif(temp_df[num_cols], temp_df['default'])[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh', title='Значимость непрерывных переменных по ANOVA F test')

***Резюме*** - оценка плательщика БКИ (score_bki) самый значимый показатель по ANOVA F test, потом кол-во отказанных заявок (declain_app_cnt) и в конце возраст (age)

### Посмотрим на значимость категориальных и бинарных переменных 

In [ ]:
label_encoder = LabelEncoder()
df['education_l'] = label_encoder.fit_transform(df['education'])

# паралельно подготовим бинарные переменные и переведем их в числовой формат
# для бинарных признаков мы будем использовать LabelEncoder
label_encoder = LabelEncoder()
for column in bin_cols:
    df[column] = label_encoder.fit_transform(df[column])
    
# тут могут быть потенциальные даталики, но мы пока не придумали как это обработать,
# потому что далее по этим меткам формируются новые фичи по get_dummies

all_cat_and_bin_cols = cat_cols+bin_cols
all_cat_and_bin_cols.remove('education')
all_cat_and_bin_cols.append('education_l')
print(all_cat_and_bin_cols)

temp_df = df[df['Train']==1]
imp_cat = pd.Series(mutual_info_classif(temp_df[all_cat_and_bin_cols], temp_df['default'], discrete_features =True), index = all_cat_and_bin_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh', title = 'Значимость бин. и категор. переменных по Mutual information test')

***Резюме*** - Самым значимым признаком по Mutual information тесту является связь заемщика с клиентами банка (sna) и давность наличия информации о заемщике (first_time), потом идет рейтинг региона (region_rating) и в конце пол (sex).

## 3. Подготовка данных к машинному обучению
---

### Категориальные признаки

In [ ]:
# реализуем метод OneHotLabels через get_dummies
df=pd.get_dummies(df, prefix=cat_cols, columns=cat_cols)

### Стандартизация

In [ ]:
# стандартизацию проводим отдельно для трейна и теста, чтобы не допустить даталиков
utils.StandardScaler_df_and_filna_0(df[df['Train']==1], num_cols)

utils.StandardScaler_df_and_filna_0(df[df['Train']==0], num_cols)

### Удаление нечисловых критериев

In [ ]:
df.drop(['app_date', 'education_l'], axis=1, inplace=True)

## 4. Построение модели
---
### Разбиваем датасет на тренировочный и тестовый

In [ ]:
train_data = df.query('Train == 1').drop(['Train', 'client_id'], axis=1)
test_data = df.query('Train == 0').drop(['Train', 'client_id'], axis=1)

y = train_data.default.values            # наш таргет
X = train_data.drop(['default'], axis=1)

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

### Создаём эксперимент в MLFLow

In [ ]:
mlflow.set_experiment('default-predict') 

### Обучаем модель, генерируем результат и сравниваем с тестом

In [ ]:
# первая модель на автомате
model = LogisticRegression(random_state=RANDOM_SEED)

model.fit(X_train, y_train)

y_pred_prob = model.predict_proba(X_test)[:,1]
y_pred = model.predict(X_test)

### Оценка качества модели
---
### Качественные метрики

In [ ]:
# в первый раз инициируем глобальную переменную с предыдущим скором
utils.last_pred = np.zeros((3,len(y_test)))

In [ ]:
utils.test_last_pred(y_test, y_pred, y_pred_prob) if (utils.last_pred[0].max() == 0) else 0
utils.all_metrics(y_test, y_pred, y_pred_prob)

### Матрица ошибок

In [ ]:
utils.confusion_matrix_f(['Дефолтный','Не дефолтный'], y_test, y_pred, 1.2, normalize=False)

### ROC кривая

In [ ]:
utils.ROC_curve_with_area(y_test, y_pred_prob, 1.1)

***Резюме*** - прекрасный пример несостоятельности метрики ROC-AUC на не сбалансированных данных. Мы абсолютно не угадали дефолтных клиентов, тем самым обеспечили себе огромную ошибку второго рода и как следствие колосальный убыток, но ROC-AUC у нас высокий. Благо f1 как-то сигнализирует о том что что-то не впорядке. Надо посмотреть на метрику которая может оценивать эффективность алгоритма на несбалансированных данных - PRC-AUC.

### Precision-Recall кривая

In [ ]:
utils.PR_curve_with_area(y_test, y_pred, 1.1)

### Сохранение метрик, артефактов и графиков в MLFLow

In [ ]:
eval_data = X_test.copy()
eval_data["target"] = y_test

with mlflow.start_run() as run:
   model_info = mlflow.sklearn.log_model(model, "model")
   result = mlflow.evaluate(
       model_info.model_uri,
       eval_data,
       targets="target",
       model_type="classifier",
       dataset_name="default-predict",
       evaluators="default",
       evaluator_config={"explainability_nsamples": 1000},
   )

### Кросс-валидация

In [ ]:
temp_vec = cross_validate(model, X_test, y_test, cv=10, scoring='roc_auc', return_train_score=True)
utils.vis_cross_val_score('ROC-AUC', temp_vec, 0.744846, 1.1)

***Резюме*** - модель, которую мы получили, очень плохая, несмотря на то, что целевая метрика ROC-AUC достаточно высокая (=0.745).  
Потому что:
- Из матрицы ошибок видно, что мы почти не угадываем дефолтных клиентов (38 из 1789). Об этом также свидетествует метрика полнота recall = 0.020799, которая собственно описывает этот момент. Таким образом все клиенты попали в ошибку второго рода, а это очень плохо для банка, так как мы будем выдавать кредиты людям которые их не смогут вернуть - риск потерять все деньги. 
- Но целевая переменная площадь под ROC кривой оказалась не такой уж плохой. Дело в том, что эта кривая плохо оценивает эффективность алгоритма на несбалансированных данных, поэтому мы добавили Precision-Recall кривую и увидели что эффективность нашего алгоритма оставляет желать лучшего.  

Кросс валидация показала дисперсию ошибки на тесте 0.014 и так как мы не использовали перемешивания, то можно сказать что модель лучше работает на крайних фолдах, чем на средних. В анализе признака data приводилась аналитика, которая говорила о том что вероятно в середине периода в банке проводилась какая-то акция и порог выдачи кредита снижался. Но мы в своей работе стремились создать надежный алгоритм который снижает ошибку второго рода (снижает потери банка) и достаточно стабильно работает (f1, PRC_AUC) при оптимальных значениях точности, а не гнались за просто высоким результатом на лидерборде.

### Поиск оптимальных параметров модели

In [ ]:
# запускаем GridSearch на небольшом кол-ве итераций max_iter=50 и с достаточно большой дельтой останова tol1e-3
# чтобы получить оптимальные параметры модели в первом приближении
model = LogisticRegression(random_state=RANDOM_SEED)

iter_ = 50
epsilon_stop = 1e-3

param_grid = [
    {'penalty': ['l1'], 
     'solver': ['liblinear', 'lbfgs'], 
     'class_weight':['none', 'balanced'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[iter_],
     'tol':[epsilon_stop]},
    {'penalty': ['l2'], 
     'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 
     'class_weight':['none', 'balanced'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[iter_],
     'tol':[epsilon_stop]},
    {'penalty': ['none'], 
     'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'], 
     'class_weight':['none', 'balanced'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[iter_],
     'tol':[epsilon_stop]},
]
gridsearch = GridSearchCV(model, param_grid, scoring='f1', n_jobs=-1, cv=5)
gridsearch.fit(X_train, y_train)
model = gridsearch.best_estimator_
##печатаем параметры
best_parameters = model.get_params()
for param_name in sorted(best_parameters.keys()):
        print('\t%s: %r' % (param_name, best_parameters[param_name]))
    ##печатаем метрики
preds = model.predict(X_test)
print('Accuracy: %.4f' % accuracy_score(y_test, preds))
print('Precision: %.4f' % precision_score(y_test, preds))
print('Recall: %.4f' % recall_score(y_test, preds))
print('F1: %.4f' % f1_score(y_test, preds))

***Резюме*** - метрика f1 подросла почти в 10 раз, что вселяет надежду на то, что модель теперь будет работать эффективнее. Построим модель на этих параметрах и посмотрим, но сначала заново пересоберем трейн и тест перемешав выборку

In [ ]:
# вторая модель на параметрах после первой оптимизации
model = LogisticRegression(random_state=RANDOM_SEED, 
                           C=1, 
                           class_weight= 'balanced', 
                           dual= False, 
                           fit_intercept= True, 
                           intercept_scaling= 1, 
                           l1_ratio= None, 
                           multi_class= 'auto', 
                           n_jobs= None, 
                           penalty= 'l2', 
                           solver = 'sag', 
                           verbose= 0, 
                           warm_start= False)

model.fit(X_train, y_train)

y_pred_prob = model.predict_proba(X_test)[:,1]
y_pred = model.predict(X_test)

### Сохраняем данные о модели в MLFLow

In [ ]:
with mlflow.start_run() as run:
   model_info = mlflow.sklearn.log_model(model, "model")
   result = mlflow.evaluate(
       model_info.model_uri,
       eval_data,
       targets="target",
       model_type="classifier",
       dataset_name="default-predict",
       evaluators="default",
       evaluator_config={"explainability_nsamples": 1000},
   )

### Оценка качества модели на оптимальных параметрах 

In [ ]:
utils.test_last_pred(y_test, y_pred, y_pred_prob) if (utils.last_pred[0].max() == 0) else 0
utils.all_metrics(y_test, y_pred, y_pred_prob)

***Резюме*** - все метрики подросли по сравнению с первой моделью, кроме accuracy и precision. Но зато сбалансированная accuracy выросла и мы можем ожидать не такой провал по ошибке второго рода как на первой модели. Ну и точность модели precision просела тоже в том числе потому что модель стала работать лучше и в FP полились клиенты из TN. Посмотрим матрицу ошибок и убедимся в этом

In [ ]:
utils.confusion_matrix_f(['Дефолтный','Не дефолтный'], y_test, y_pred, 1.2, normalize=False)

In [ ]:
utils.ROC_curve_with_area(y_test, y_pred_prob, 1.1)

In [ ]:
utils.PR_curve_with_area(y_test, y_pred, 1.1)

***Резюме*** - эффективность алгоритма подросла и теперь более менее хорошо ведет себя на несбалансированных данных. Проверим это с помощью кросс-валидации

### Кросс-валидация

In [ ]:
temp_vec = cross_validate(model, X_test, y_test, cv=10, scoring='roc_auc', return_train_score=True)
utils.vis_cross_val_score('ROC-AUC', temp_vec, 0.744223, 1.1)

Резюме - Видно что при разбиении на 10 фолдов ROC-AUC меняется не сильно, есть один провал до 0.716. Среднее по фолдам примерно равно предсказанному значению до кроссвалидации 0.744, дисперсия 0.015, поэтому можно попробовать докрутить параметры и провести отбор признаков. Также можно попробовать прокрутить модель на регуляризации первого порядка l1 и попробовать избавиться от каких-нибудь признаков, которые не нравятся модели.

In [ ]:
{
  "data": {
    "names": ["sex", "age", "car", "car_type", "decline_app_cnt", "good_work",
       "score_bki", "bki_request_cnt", "income", "foreign_passport", "days",
       "education_ACD", "education_GRD", "education_PGR", "education_SCH",
       "education_UGR", "region_rating_20", "region_rating_30",
       "region_rating_40", "region_rating_50", "region_rating_60",
       "region_rating_70", "region_rating_80", "home_address_1",
       "home_address_2", "home_address_3", "work_address_1", "work_address_2",
       "work_address_3", "sna_1", "sna_2", "sna_3", "sna_4", "first_time_1",
       "first_time_2", "first_time_3", "first_time_4"],
    "tensor": {
      "shape": [
        1,
        37
      ],
      "values": [
 0,
 3.4011973816621555,
 0,
 0,
 0.6931471805599453,
 0,
 1.19598062,
 1.3862943611198906,
 10.043292972227004,
 0,
 73,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1
      ]
    }
  }
}

In [ ]:
raise SystemExit("Stop right there!")

# Feast Demo

###  Установка библиотек

In [ ]:
pip install feast==0.23.0 feast-cassandra psycopg2-binary

### Настройка окружения

In [ ]:
import os
import warnings

os.environ["FEAST_S3_ENDPOINT_URL"]="http://ceph01.do.neoflex.ru:8080"
warnings.filterwarnings('ignore')

In [ ]:
!rm -rf dags

### Создание FeatureView на основе данных кредитной истории клиента

In [ ]:
from datetime import timedelta

from feast import (Entity, Feature, FeatureView, ValueType, FeatureService)
from feast.infra.offline_stores.contrib.postgres_offline_store.postgres_source import (
    PostgreSQLSource,
)

zipcode = Entity(name="zipcode", value_type=ValueType.INT64)

zipcode_source = PostgreSQLSource(
    name="zipcode",
    query="SELECT * FROM zipcode_table",
    timestamp_field="event_timestamp",
    created_timestamp_column="created_timestamp",
)

zipcode_features = FeatureView(
    name="zipcode_features",
    entities=["zipcode"],
    ttl=timedelta(days=3650),
    features=[
        Feature(name="city", dtype=ValueType.STRING),
        Feature(name="state", dtype=ValueType.STRING),
        Feature(name="location_type", dtype=ValueType.STRING),
        Feature(name="tax_returns_filed", dtype=ValueType.INT64),
        Feature(name="population", dtype=ValueType.INT64),
        Feature(name="total_wages", dtype=ValueType.INT64),
    ],
    batch_source=zipcode_source,
)

dob_ssn = Entity(
    name="dob_ssn",
    value_type=ValueType.STRING,
    description="Date of birth and last four digits of social security number",
)

credit_history_source = PostgreSQLSource(
    name="credit_history",
    query="SELECT * FROM credit_history",
    timestamp_field="event_timestamp",
    created_timestamp_column="created_timestamp",
)

credit_history = FeatureView(
    name="credit_history",
    entities=["dob_ssn"],
    ttl=timedelta(days=300),
    features=[
        Feature(name="credit_card_due", dtype=ValueType.INT64),
        Feature(name="mortgage_due", dtype=ValueType.INT64),
        Feature(name="student_loan_due", dtype=ValueType.INT64),
        Feature(name="vehicle_loan_due", dtype=ValueType.INT64),
        Feature(name="hard_pulls", dtype=ValueType.INT64),
        Feature(name="missed_payments_2y", dtype=ValueType.INT64),
        Feature(name="missed_payments_1y", dtype=ValueType.INT64),
        Feature(name="missed_payments_6m", dtype=ValueType.INT64),
        Feature(name="bankruptcies", dtype=ValueType.INT64),
    ],
    batch_source=credit_history_source,
)

feature_service = FeatureService(
    name='loan_features',
    features=[zipcode_features, credit_history]
)

In [ ]:
# try with feast-hive
features = f"""
from datetime import timedelta

from feast import (Entity, Feature, FeatureView, ValueType, FeatureService)
from feast.infra.offline_stores.contrib.postgres_offline_store.postgres_source import (
    PostgreSQLSource,
)

zipcode = Entity(name="zipcode", value_type=ValueType.INT64)

zipcode_source = PostgreSQLSource(
    name="zipcode",
    query="SELECT * FROM zipcode_table",
    timestamp_field="event_timestamp",
    created_timestamp_column="created_timestamp",
)

zipcode_features = FeatureView(
    name="zipcode_features",
    entities=["zipcode"],
    ttl=timedelta(days=3650),
    features=[
        Feature(name="city", dtype=ValueType.STRING),
        Feature(name="state", dtype=ValueType.STRING),
        Feature(name="location_type", dtype=ValueType.STRING),
        Feature(name="tax_returns_filed", dtype=ValueType.INT64),
        Feature(name="population", dtype=ValueType.INT64),
        Feature(name="total_wages", dtype=ValueType.INT64),
    ],
    batch_source=zipcode_source,
)

dob_ssn = Entity(
    name="dob_ssn",
    value_type=ValueType.STRING,
    description="Date of birth and last four digits of social security number",
)

credit_history_source = PostgreSQLSource(
    name="credit_history",
    query="SELECT * FROM credit_history",
    timestamp_field="event_timestamp",
    created_timestamp_column="created_timestamp",
)

credit_history = FeatureView(
    name="credit_history",
    entities=["dob_ssn"],
    ttl=timedelta(days=200),
    features=[
        Feature(name="credit_card_due", dtype=ValueType.INT64),
        Feature(name="mortgage_due", dtype=ValueType.INT64),
        Feature(name="student_loan_due", dtype=ValueType.INT64),
        Feature(name="vehicle_loan_due", dtype=ValueType.INT64),
        Feature(name="hard_pulls", dtype=ValueType.INT64),
        Feature(name="missed_payments_2y", dtype=ValueType.INT64),
        Feature(name="missed_payments_1y", dtype=ValueType.INT64),
        Feature(name="missed_payments_6m", dtype=ValueType.INT64),
        Feature(name="bankruptcies", dtype=ValueType.INT64),
    ],
    batch_source=credit_history_source,
)

feature_service = FeatureService(
    name='loan_features',
    features=[zipcode_features, credit_history]
)

"""
with open('features.py', "w") as features_file:
    features_file.write(features)

In [ ]:
!cp feast_gazprom/feature_store.yaml .

In [ ]:
!feast apply

#### Batch Demo

In [ ]:
import pandas as pd
data_for_pg = X_test
train_data_for_pg = df_train
test_data_for_pg = df_test

In [ ]:
X_test

In [ ]:
pip install psycopg2-binary SQLAlchemy

In [ ]:
DATABASE = {
    'drivername': 'postgresql+psycopg2', #Тут можно использовать MySQL или другой драйвер
    'host': 'cassandra-postgresql.feast-db',
    'port': '5432',
    'username': 'postgres',
    'password': 'postgres',
    'database': 'FEAST_OFFLINE_STORE'
}

from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
engine  = create_engine(URL(**DATABASE))

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
engine  = create_engine(URL(**DATABASE))

In [ ]:
#data_for_pg.to_sql('data', engine)
train_data_for_pg.to_sql('train',engine)
test_data_for_pg.to_sql('test',engine)

In [ ]:
import psycopg2
conn = psycopg2.connect("host=cassandra-postgresql.feast-db port=5432 dbname=FEAST_OFFLINE_STORE user=postgres password=postgres")
cur = conn.cursor()
cur.execute('SELECT * FROM data')
cur.fetchone()

In [ ]:
df = pd.read_sql_query('SELECT * FROM data',con=engine)

In [ ]:
data = df.drop(['index'], axis=1)

In [ ]:
data

In [ ]:
import mlflow.sklearn
model = mlflow.sklearn.load_model("models:/default-predict/1")

In [ ]:
ypred = pd.DataFrame({'predicts': pd.Series(model.predict(data))})

In [ ]:
df_total = pd.concat([data, ypred], axis=1)

In [ ]:
df_total

In [ ]:
df_total.to_sql('predictions', engine)

In [ ]:
cur.execute('SELECT * FROM predictions')
cur.fetchone()

In [ ]:
DATABASE = {
    'drivername': 'postgresql+psycopg2', #Тут можно использовать MySQL или другой драйвер
    'host': 'cassandra-postgresql.feast-db',
    'port': '5432',
    'username': 'postgres',
    'password': 'postgres',
    'database': 'FEAST_OFFLINE_STORE'
}

from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
engine  = create_engine(URL(**DATABASE))

df = pd.read_sql_query('SELECT * FROM data',con=engine)

data = df.drop(['index'], axis=1)

In [ ]:
import mlflow.sklearn
model = mlflow.sklearn.load_model("models:/default-predict/1")

ypred = pd.DataFrame({'predicts': pd.Series(model.predict(data))})
df_total = pd.concat([data, ypred], axis=1)

In [ ]:
DATABASE = {
    'drivername': 'postgresql+psycopg2', #Тут можно использовать MySQL или другой драйвер
    'host': 'cassandra-postgresql.feast-db',
    'port': '5432',
    'username': 'postgres',
    'password': 'postgres',
    'database': 'FEAST_OFFLINE_STORE'
}

from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
engine  = create_engine(URL(**DATABASE))

df_total.to_sql('predictions', engine)

In [ ]:
def predict(**kwargs):
    DATABASE = {
        'drivername': 'postgresql+psycopg2', #Тут можно использовать MySQL или другой драйвер
        'host': 'dognauts-demo-mlflow-postgresql',
        'port': '5432',
        'username': 'postgres',
        'password': 'mlflow123',
        'database': 'mlflowdb'
    }

    from sqlalchemy import create_engine
    from sqlalchemy.engine.url import URL
    engine  = create_engine(URL(**DATABASE))

    df = pd.read_sql_query('SELECT * FROM data',con=engine)

    data = df.drop(['index'], axis=1)

    import mlflow.sklearn
    model = mlflow.sklearn.load_model("models:/default-predict/1")

    ypred = pd.DataFrame({'predicts': pd.Series(model.predict(data))})
    df_total = pd.concat([data, ypred], axis=1)

    df_total.to_sql('predictions', engine)